In [1]:
# import dependencies
import pandas as pd
import sqlite3
from datetime import datetime, timedelta
from fbprophet import Prophet

from fbprophet.plot import plot_plotly
import plotly.offline as py

import plotly.offline as py
import plotly.graph_objs as go

In [12]:
# Create your connections with indoor and outdoor databases
Ocnx = sqlite3.connect('Outdoor.db')
Icnx = sqlite3.connect('Indoor.db')
Outdoor_df = pd.read_sql_query("SELECT * FROM BME_DATA", Ocnx)
Indoor_df = pd.read_sql_query("SELECT * FROM BME_DATA", Icnx)

In [13]:
# Convert time into datetime
Outdoor_df['TIME_STAMP'] = pd.to_datetime(Outdoor_df['TIME_STAMP'])
Indoor_df['TIME_STAMP'] = pd.to_datetime(Indoor_df['TIME_STAMP'])

In [14]:
Outdoor_df['TIME_STAMP'] = Outdoor_df['TIME_STAMP'].dt.round('60min')
Indoor_df['TIME_STAMP'] = Indoor_df['TIME_STAMP'].dt.round('60min')
Outdoor_df.head()

,id,TIME_STAMP,TEMPERATURE,GAS,HUMIDITY,PRESSURE,ALTITUDE
0,16882,2019-05-24 13:00:00,31.5,723933,54.8,989.5,199.3


In [5]:
Indoor_df.head()

,id,TIME_STAMP,TEMPERATURE,GAS,HUMIDITY,PRESSURE,ALTITUDE
0,1,2019-05-12 19:00:00,75.0,100,50.0,1000.0,1000.0
1,2,2019-05-12 19:00:00,23.5,6049661,35.9,972.9,341.2
2,3,2019-05-12 19:00:00,23.5,5851528,35.9,972.9,341.3
3,4,2019-05-12 19:00:00,23.4,5636172,36.2,972.9,341.4
4,5,2019-05-12 19:00:00,23.5,5441624,37.3,972.9,341.5


In [6]:
# Deleting the wrong input at the first rows
Outdoor_df = Outdoor_df.iloc[1:]
Indoor_df = Indoor_df.iloc[1:]

In [7]:
# converting temperature from C to f
Outdoor_df['TEMPERATURE'] = round((Outdoor_df['TEMPERATURE']* 9/5) + 32)
Indoor_df['TEMPERATURE'] = round((Indoor_df['TEMPERATURE']* 9/5) + 32)

In [8]:
# Merging Indor and Outdoor (on time stamp)
master_df = pd.merge(Outdoor_df, Indoor_df, on = "TIME_STAMP", how = "left", suffixes=("_Out","_In"))

In [9]:
# Dropping NAs
master_df= master_df.dropna()
master_df.head()

,id_Out,TIME_STAMP,TEMPERATURE_Out,GAS_Out,HUMIDITY_Out,PRESSURE_Out,ALTITUDE_Out,id_In,TEMPERATURE_In,GAS_In,HUMIDITY_In,PRESSURE_In,ALTITUDE_In
0,2,2019-05-12 19:00:00,75.0,364884,84.9,975.0,323.2,2,74.0,6049661,35.9,972.9,341.2
1,2,2019-05-12 19:00:00,75.0,364884,84.9,975.0,323.2,3,74.0,5851528,35.9,972.9,341.3
2,2,2019-05-12 19:00:00,75.0,364884,84.9,975.0,323.2,4,74.0,5636172,36.2,972.9,341.4
3,2,2019-05-12 19:00:00,75.0,364884,84.9,975.0,323.2,5,74.0,5441624,37.3,972.9,341.5
4,2,2019-05-12 19:00:00,75.0,364884,84.9,975.0,323.2,6,74.0,5219085,37.1,972.9,341.6


In [10]:
# Creating Json for chart1
prep_json = master_df.loc[:,["TIME_STAMP", "TEMPERATURE_Out","TEMPERATURE_In"]]
prep_json2 = prep_json.groupby(['TIME_STAMP'], as_index=False)["TEMPERATURE_Out","TEMPERATURE_In"].mean()
prep_json2["TEMPERATURE_Out"] = round(prep_json2["TEMPERATURE_Out"],1)
json_df = prep_json2.to_json(orient='records')

In [11]:
json_df

'[{"TIME_STAMP":1557687600000,"TEMPERATURE_Out":74.6,"TEMPERATURE_In":74.5},{"TIME_STAMP":1557691200000,"TEMPERATURE_Out":73.0,"TEMPERATURE_In":74.5333333333},{"TIME_STAMP":1557694800000,"TEMPERATURE_Out":71.7,"TEMPERATURE_In":74.0},{"TIME_STAMP":1557698400000,"TEMPERATURE_Out":71.9,"TEMPERATURE_In":74.6333333333},{"TIME_STAMP":1557702000000,"TEMPERATURE_Out":71.4,"TEMPERATURE_In":74.7},{"TIME_STAMP":1557705600000,"TEMPERATURE_Out":70.4,"TEMPERATURE_In":75.9333333333},{"TIME_STAMP":1557709200000,"TEMPERATURE_Out":70.0,"TEMPERATURE_In":76.0},{"TIME_STAMP":1557712800000,"TEMPERATURE_Out":69.9,"TEMPERATURE_In":76.0},{"TIME_STAMP":1557716400000,"TEMPERATURE_Out":69.0,"TEMPERATURE_In":76.0},{"TIME_STAMP":1557720000000,"TEMPERATURE_Out":67.8,"TEMPERATURE_In":76.7333333333},{"TIME_STAMP":1557723600000,"TEMPERATURE_Out":66.4,"TEMPERATURE_In":76.6166666667},{"TIME_STAMP":1557727200000,"TEMPERATURE_Out":66.0,"TEMPERATURE_In":76.0},{"TIME_STAMP":1557730800000,"TEMPERATURE_Out":65.2,"TEMPERATURE_I

In [ ]:
# Creating indoor and Outdoor Datasets
outdoor = master_df.loc[:,["TIME_STAMP", "TEMPERATURE_Out"]]
indoor = master_df.loc[:,["TIME_STAMP", "TEMPERATURE_In"]]
outdoor.columns = ['ds', 'y']
indoor.columns = ['ds', 'y']

In [ ]:
# Export to csv
outdoor.to_csv("fbp.csv", sep=',', encoding='utf-8')

In [ ]:
# Grouping by Hour
outdoor = outdoor.groupby(['ds'], as_index=False).y.mean()
indoor = indoor.groupby(['ds'], as_index=False).y.mean()

In [ ]:
indoor.head()

In [ ]:
# Building the model for inodor and outdoor
od = Prophet()
od.fit(outdoor)
ind = Prophet()
ind.fit(indoor)

In [ ]:
# Outdoor forecasting
od_future = od.make_future_dataframe(periods= 1)
od_forecast = od.predict(od_future)
# Export to csv
od_forecast.to_csv("od_forecast.csv", sep=',', encoding='utf-8')
od_forecast.head()

In [ ]:
# Indoor forecasting
ind_future = ind.make_future_dataframe(periods= 1)
ind_forecast = ind.predict(ind_future)
ind_forecast.to_csv("ind_forecast.csv", sep=',', encoding='utf-8')
ind_forecast.head()

In [ ]:
# Outdoor plot with forecasting
od_plot = od.plot(od_forecast)

In [ ]:
# Indoor plot with forecasting
ind_plot = ind.plot(ind_forecast)

In [ ]:
# Print outdoor trend and daily component
od_fig1 = od.plot_components(od_forecast)

In [ ]:
# Print indoor trend and daily component
ind_fig1 = ind.plot_components(ind_forecast)

In [ ]:
# Print outdoor 
py.init_notebook_mode()
od_fig = plot_plotly(od, od_forecast)  # This returns a plotly Figure
py.iplot(od_fig)

In [ ]:
# Print indoor plotly figure
ind_fig = plot_plotly(ind, ind_forecast)  # This returns a plotly Figure
py.iplot(ind_fig)

In [ ]:
# Producing full plot for outdoor data
py.iplot([
    go.Scatter(x=outdoor['ds'], y=outdoor['y'], name='y'),
    go.Scatter(x=od_forecast['ds'], y=od_forecast['yhat'], name='yhat'),
    go.Scatter(x=od_forecast['ds'], y=od_forecast['yhat_upper'], fill='tonexty', mode='none', name='upper'),
    go.Scatter(x=od_forecast['ds'], y=od_forecast['yhat_lower'], fill='tonexty', mode='none', name='lower'),
    go.Scatter(x=od_forecast['ds'], y=od_forecast['trend'], name='Trend')
])

In [ ]:
# Producing full plot for indoor data
py.iplot([
    go.Scatter(x=indoor['ds'], y=indoor['y'], name='y'),
    go.Scatter(x=ind_forecast['ds'], y=ind_forecast['yhat'], name='yhat'),
    go.Scatter(x=ind_forecast['ds'], y=ind_forecast['yhat_upper'], fill='tonexty', mode='none', name='upper'),
    go.Scatter(x=ind_forecast['ds'], y=ind_forecast['yhat_lower'], fill='tonexty', mode='none', name='lower'),
    go.Scatter(x=ind_forecast['ds'], y=ind_forecast['trend'], name='Trend')
])